# 🧠 Topic Modeling with LDA — Using `df_preprocessing.csv` Dataset

Notebook ini membangun **Latent Dirichlet Allocation (LDA)** untuk dataset `df_preprocessing.csv`, yang memiliki kolom:
- `tokenize_indo` → teks berita hasil preprocessing/tokenisasi
- `Kategori Berita` → label kategori berita (opsional, untuk analisis perbandingan)

In [3]:
import pandas as pd
import numpy as np
import re
# import nltk
# from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# nltk.download('stopwords')

## 1️⃣ Load Dataset

In [4]:
# Load dataset
df = pd.read_csv('datasets/df_preprocessing.csv')

# Ambil kolom yang sudah dipreprocessing
documents = df['tokenize_indo'].astype(str).tolist()
labels = df['Kategori Berita'].tolist()

print('Total documents:', len(documents))
df.head()

Total documents: 1200


,Isi Berita,lwr_indo,clean_sw_indo,clean_stb_indo,clean_typo_indo,stemming_indo,tokenize_indo,Kategori Berita
0,KOMPAS.com - Menteri Pemuda dan Olahraga Erick...,kompas.com - menteri pemuda dan olahraga erick...,kompas.com - menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick tho...,kompas com menteri pemuda olahraga erick thohi...,kompas com menteri pemuda olahraga erick thohi...,"['kompas', 'com', 'menteri', 'pemuda', 'olahra...",BOLA
1,"KOMPAS.com - Manajer Chelsea, Enzo Maresca men...","kompas.com - manajer chelsea, enzo maresca men...","kompas.com - manajer chelsea, enzo maresca men...",kompas com manajer chelsea enzo maresca meng...,kompas com manajer chelsea enzo maresca mengan...,kompas com manajer chelsea enzo maresca anggap...,"['kompas', 'com', 'manajer', 'chelsea', 'enzo'...",BOLA
2,"KOMPAS.com - Pelatih Liverpool, Arne Slot, men...","kompas.com - pelatih liverpool, arne slot, men...","kompas.com - pelatih liverpool, arne slot, men...",kompas com pelatih liverpool arne slot menga...,kompas com pelatih liverpool arne slot mengaku...,kompas com latih liverpool arne slot aku cryst...,"['kompas', 'com', 'latih', 'liverpool', 'arne'...",BOLA
3,KOMPAS.com - Hasil terbaru pekan kelima Liga I...,kompas.com - hasil terbaru pekan kelima liga i...,kompas.com - hasil terbaru pekan liga italia 2...,kompas com hasil terbaru pekan liga italia ...,kompas com hasil terbaru pekan liga italia per...,kompas com hasil baru pekan liga italia beda n...,"['kompas', 'com', 'hasil', 'baru', 'pekan', 'l...",BOLA
4,KOMPAS.com - Menteri Pemuda dan Olahraga Repub...,kompas.com - menteri pemuda dan olahraga repub...,kompas.com - menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik ...,kompas com menteri pemuda olahraga republik in...,kompas com menteri pemuda olahraga republik in...,"['kompas', 'com', 'menteri', 'pemuda', 'olahra...",BOLA


## 2️⃣ Text Normalization & Preprocessing

In [5]:
# stop_words = set(stopwords.words('indonesian'))

# def normalize_text(text):
#     text = text.lower()
#     text = re.sub(r'[^a-zA-Z\s]', '', text)
#     tokens = text.split()
#     tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
#     return ' '.join(tokens)

# norm_corpus = [normalize_text(doc) for doc in documents]
# print('Contoh hasil normalisasi:')
# print(norm_corpus[0][:200])

## 3️⃣ Text Vectorization

In [6]:
cv = CountVectorizer(max_df=0.9, min_df=5, stop_words='english')
cv_matrix = cv.fit_transform(documents)
print('Shape of document-term matrix:', cv_matrix.shape)
vocab = cv.get_feature_names_out()

Shape of document-term matrix: (1200, 4168)


## 4️⃣ Train LDA Model

In [7]:
lda = LatentDirichletAllocation(n_components=5, max_iter=1000, random_state=0)
doc_topic_matrix = lda.fit_transform(cv_matrix)
print('LDA model trained!')

LDA model trained!


## 5️⃣ Inspect Topics

In [8]:
def get_topics_meanings(components, feature_names, n_top_words=10):
    topics = {}
    for idx, topic in enumerate(components):
        top_features = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics[f'Topic {idx+1}'] = top_features
    return topics

topics = get_topics_meanings(lda.components_, vocab, n_top_words=10)
for k, v in topics.items():
    print(f'{k}:', ', '.join(v))

Topic 1: mobil, kendara, motor, listrik, milik, bbm, indonesia, jakarta, honda, jalan
Topic 2: mbg, makan, racun, siswa, sekolah, anak, program, gizi, dapur, barat
Topic 3: korban, jalan, warga, rumah, orang, sebut, itu, jakarta, laku, polisi
Topic 4: rp, indonesia, jakarta, kerja, usaha, persen, menteri, perintah, harga, negara
Topic 5: main, laga, indonesia, gol, vs, hasil, tim, menang, liga, menit


## 6️⃣ pyLDAvis Visualization

In [9]:
import pyLDAvis
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, cv_matrix, cv, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.140149 -0.099406       1        1  27.613095
3     -0.005515  0.199647       2        1  25.450057
4     -0.129274 -0.252393       3        1  19.456323
0     -0.227927  0.080874       4        1  14.655884
1      0.222567  0.071277       5        1  12.824641, topic_info=          Term        Freq       Total Category  logprob  loglift
2302       mbg  699.000000  699.000000  Default  30.0000  30.0000
2199      main  837.000000  837.000000  Default  29.0000  29.0000
2391     mobil  624.000000  624.000000  Default  28.0000  28.0000
2204     makan  500.000000  500.000000  Default  27.0000  27.0000
1766   kendara  631.000000  631.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
2786  perintah  136.916203  708.181294   Topic5  -5.3227   0.4105
2601     orang  111.889941  550.877853   Topic5  -5.5245   0.4598
2026  langsung  106.148829  465.790704   Topic5  -5.5772   0.5749
1487       itu  116.824090  941.507886   Topic5  -5.4814  -0.0330
3302     sebut  108.541592  777.844081   Topic5  -5.5549   0.0844

[334 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
65        4  0.994381       ai
72        1  0.200313     ajar
72        3  0.061207     ajar
72        4  0.033385     ajar
72        5  0.701094     ajar
...     ...       ...      ...
4125      2  0.169546  wilayah
4125      3  0.007886  wilayah
4125      5  0.500753  wilayah
4135      5  0.990744      wuk
4147      4  0.987962   yamaha

[621 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 1, 2])

## 7️⃣ Cluster Documents Based on Topic Distributions

In [10]:
km = KMeans(n_clusters=5, random_state=0)
km.fit(doc_topic_matrix)
df['ClusterLabel'] = km.labels_
df[['tokenize_indo', 'ClusterLabel']].head()

,tokenize_indo,ClusterLabel
0,"['kompas', 'com', 'menteri', 'pemuda', 'olahra...",2
1,"['kompas', 'com', 'manajer', 'chelsea', 'enzo'...",3
2,"['kompas', 'com', 'latih', 'liverpool', 'arne'...",3
3,"['kompas', 'com', 'hasil', 'baru', 'pekan', 'l...",3
4,"['kompas', 'com', 'menteri', 'pemuda', 'olahra...",2


## 8️⃣ Hyperparameter Tuning (Grid Search)

In [ ]:
search_params = {'n_components': range(3,8), 'learning_decay': [.5, .7]}
model = LatentDirichletAllocation(learning_method='batch', max_iter=1000, random_state=0)
gridsearch = GridSearchCV(model, param_grid=search_params, n_jobs=-1, verbose=1)
gridsearch.fit(cv_matrix)
best_lda = gridsearch.best_estimator_

print('Best Model Params:', gridsearch.best_params_)
print('Best Log Likelihood:', gridsearch.best_score_)
print('Best Perplexity:', best_lda.perplexity(cv_matrix))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


## 9️⃣ Predict Topics for New Text

In [ ]:
new_docs = ['pemerintah mengumumkan kebijakan ekonomi baru',
            'tim sepak bola memenangkan pertandingan final']
new_docs_norm = [normalize_text(t) for t in new_docs]
new_docs_vec = cv.transform(new_docs_norm)

new_topic_matrix = best_lda.transform(new_docs_vec)
topic_labels = [f'Topic {i+1}' for i in range(best_lda.n_components)]
new_df = pd.DataFrame(new_topic_matrix, columns=topic_labels)
new_df['predicted_topic'] = new_df.idxmax(axis=1)
new_df['document'] = new_docs
new_df

## 🔟 Optional: Coherence Evaluation (tmtoolkit)

In [ ]:
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim

def topic_model_coherence_generator(topic_num_start=2, topic_num_end=8, norm_corpus='', cv_matrix='', cv=''):
    norm_corpus_tokens = [doc.split() for doc in norm_corpus]
    models = []
    coherence_scores = []

    for i in range(topic_num_start, topic_num_end):
        cur_lda = LatentDirichletAllocation(n_components=i, max_iter=1000, random_state=0)
        cur_lda.fit(cv_matrix)
        cur_coherence = metric_coherence_gensim(
            measure='c_v', top_n=10, topic_word_distrib=cur_lda.components_,
            dtm=cv.fit_transform(norm_corpus), vocab=np.array(cv.get_feature_names_out()), texts=norm_corpus_tokens)
        models.append(cur_lda)
        coherence_scores.append(np.mean(cur_coherence))
    return models, coherence_scores

models, scores = topic_model_coherence_generator(2,8,norm_corpus,cv_matrix,cv)
pd.DataFrame({'n_topics': range(2,8), 'coherence': scores})